# Warming Up - Contextualized Topic Models


---
![ctm_both.jpeg](img/ctm_both.jpeg)

### 참고
- Contextualized Topic Models
    - https://github.com/MilaNLProc/contextualized-topic-models
- 1. 문맥을 반영한 토픽 모델(Contextualized Topic Models)
    - "문맥을 반영한 토픽 모델(Contextualized Topic Models)은 문맥을 반영한 BERT의 문서 임베딩의 표현력과 기존 토픽 모델의 비지도 학습 능력을 결합하여 문서에서 주제를 가져오는 토픽 모델을 말합니다. 그리고 여기서 소개할 복합 토픽 모델(Combined Topic Models, CTM)은 문맥을 반영한 토픽 모델의 일종"
    - https://wikidocs.net/161310

# 1. 환경 셋업

In [1]:
# ! pip install contextualized-topic-models==2.2.0
# ! pip install pyldavis


# 2. 데이터 다운로드 (뉴스)

In [2]:
%%capture
!wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt

In [3]:
!head -n 5 2016-10-20.txt


 
19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성

# 3. 데이터 전처리

In [4]:
text_file = "2016-10-20.txt"

In [5]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm

In [6]:
documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()]

In [7]:
documents[:5]

['',
 '19  1990  52 1 22',
 '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽

In [8]:
preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████| 30091/30091 [00:00<00:00, 150634.53it/s]


In [9]:
print("# of preprocessed_documents: ",  len(preprocessed_documents))
print("\npreprocessed_documents: \n",  preprocessed_documents[0:3])

# of preprocessed_documents:  27540

preprocessed_documents: 
 ['오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 

# 4.BOW (Bag of Word) 임베딩 생성

In [10]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [11]:
custom_tokenizer = CustomTokenizer(Mecab())

In [12]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

아래의 훈련 결과는 train_bow_embeddings (27540, 3000) 얻음.
- 27540: Doc 의 갯수
- 3000: Vocab Size

In [13]:
%%time

train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

CPU times: user 43.7 s, sys: 64.2 ms, total: 43.8 s
Wall time: 43.8 s


In [14]:
print(train_bow_embeddings.shape)

(27540, 3000)


In [15]:
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

print("# of vocab" , len(vocab))
print(vocab[0:300])


# of vocab 3000
['00', '000', '01', '02', '10', '100', '1000', '101', '11', '119', '12', '120', '13', '14', '15', '150', '16', '17', '18', '19', '20', '200', '2000', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '300', '3000', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '400', '4000', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '500', '5000', '51', '52', '53', '54', '55', '57', '58', '59', '60', '600', '6000', '61', '62', '63', '64', '65', '66', '67', '68', '70', '700', '7000', '71', '721', '74', '7470', '75', '781', '80', '800', '8000', '82', '84', '85', '87', '90', '900', '9000', '9711', '가격', '가계', '가구', '가까운', '가까이', '가깝', '가능', '가득', '가량', '가방', '가사', '가수', '가스', '가슴', '가습기', '가온', '가운데', '가을', '가이드라인', '가입', '가장', '가정', '가졌', '가족', '가지', '가진', '가치', '각각', '각자', '각종', '간담회', '간사', '갈등', '갈치', '감각', '감독', '감동',

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# 5. BERT Output 임베딩 얻기

In [16]:
%%time

train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents, \
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

CPU times: user 1min 57s, sys: 21 s, total: 2min 18s
Wall time: 1min 9s


결과는 train_contextualized_embeddings (27540, 768) 임.
- 27540 : Doc 의 갯수
- 768: 임베딩 디멘션

In [17]:
train_contextualized_embeddings.shape

(27540, 768)

# 6. Contextualized Topic Models (CTM) 생성



In [18]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"

qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)


In [34]:
num_topic = 20
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components= num_topic, num_epochs=20)
ctm.fit(training_dataset)

Epoch: [20/20]	 Seen Samples: [550800/550800]	Train Loss: 854.9706226312863	Time: 0:00:04.170545: : 20it [01:25,  4.27s/it]


# 7. CTM 분석

50 개의 토픽을 대표하는 단어 개수(예: 5) 보여주기

In [35]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['출시', '제공', '기기', '제품', '고객'],
             1: ['패션', '디자인', '2017', '신진', '디자이너'],
             2: ['지역', '지원', '사업', '계획', '교육'],
             3: ['민주당', '대표', '후보', '트럼프', '힐러리'],
             4: ['마포구', '포토', '속보', '케이스', '중구'],
             5: ['한다면', '단순', '충분히', '균형', '통해서'],
             6: ['기업', '정부', '시장', '네이버', '사업'],
             7: ['에서', '한다', '영화', '부터', '관객'],
             8: ['제작', '남자', '사랑', '수애', '발표회'],
             9: ['북한', '미국', '미사일', '장관', '국방'],
             10: ['국회', '수석', '민정', '청와대', '국정'],
             11: ['경찰', '습니다', '터널', '경찰관', '경찰서'],
             12: ['연비', '선고', '소송', '혐의', '판사'],
             13: ['으로', '에서', '지만', '습니다', '한다'],
             14: ['공개', '뮤직', '데뷔', '그룹', '음원'],
             15: ['에게', '모습', '방송', '사람', '자신'],
             16: ['으로', '투자', '보다', '최근', '주가'],
             17: ['충분히', '균형', '통해서', '한다면', '한계'],
             18: ['균형', '한다면', '충분히', '한계', '단순'],
             19: ['영상', '뉴시스', '사진'

## 가시화

In [36]:
import pyLDAvis as vis

lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)

ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

Sampling: [10/10]: : 10it [00:28,  2.80s/it]


# 8 추론

In [37]:
import numpy as np

In [38]:
topics_predictions = ctm.get_thetas(training_dataset, n_samples=5) # get all the topic predictions

Sampling: [5/5]: : 5it [00:14,  2.93s/it]


In [39]:
print("topics_predictions :", topics_predictions.shape)

topics_predictions : (27540, 20)


# 커널 리스타팅

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)